In [2]:
# Tratar de inputar los precios que faltan por expresiones regulares
# Eliminar todas filas que todo es nulo (check)
# Checar el tema de las superficies
# Precios: unificar todos los precios a dolares
# Hacer la multiplicacion de price por la superficie
# Imputar superficies nulas por medias condicionadas por barrio o cantidad de rooms

# Importacion de librerias utilizadas

In [425]:
import pandas as pd
import numpy as np
import re

# Carga de la data de Properatti como un DataFrame de Pandas

In [382]:
data_location = "../Data/properatti.csv"

data = pd.read_csv(data_location)

data.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [383]:
# Eliminamos la columna Unnamed dado que son los mismos indices
data = data.drop(['Unnamed: 0'], axis=1)
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


# Limpieza de datos

## Analizamos que state_name vamos a utilizar

In [384]:
# Vemos todos los diferentes state_name que hay en el DataSet
data.state_name.unique()

array(['Capital Federal', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Costa Atlántica', 'Entre Ríos',
       'Bs.As. G.B.A. Zona Norte', 'Santa Fe', 'Córdoba',
       'Bs.As. G.B.A. Zona Oeste', 'Misiones', 'Buenos Aires Interior',
       'Salta', 'Neuquén', 'Río Negro', 'San Luis', 'Mendoza',
       'Corrientes', 'Chubut', 'Tucumán', 'La Pampa', 'Chaco', 'San Juan',
       'Santa Cruz', 'Tierra Del Fuego', 'Catamarca',
       'Santiago Del Estero', 'Jujuy', 'La Rioja', 'Formosa'],
      dtype=object)

In [385]:
# Vemos la cantidad de instancias que tiene cada state_name
state_name_distribution = pd.value_counts(data.state_name)
state_name_distribution

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

In [386]:
# Vemos los porcentajes de la districion de los datos por provincia
total_data = data.shape[0]
percentage_state_name = 100*state_name_distribution/total_data
percentage_state_name

Capital Federal                 26.658967
Bs.As. G.B.A. Zona Norte        21.085629
Bs.As. G.B.A. Zona Sur          11.509652
Córdoba                          9.956278
Santa Fe                         8.391355
Buenos Aires Costa Atlántica     8.254413
Bs.As. G.B.A. Zona Oeste         7.690150
Buenos Aires Interior            1.889952
Río Negro                        0.666557
Neuquén                          0.604686
Mendoza                          0.561788
Tucumán                          0.556014
Corrientes                       0.480944
Misiones                         0.382775
Entre Ríos                       0.304405
Salta                            0.229335
Chubut                           0.213661
San Luis                         0.207886
La Pampa                         0.129517
Formosa                          0.053622
Chaco                            0.047022
San Juan                         0.032998
Tierra Del Fuego                 0.025573
Catamarca                        0

In [387]:
# Tomamos la decision de quedarnos con los datos que son > 1%, que corresponden a Buenos Aires Interior paara arriba
mask_1_percent = percentage_state_name[percentage_state_name > 1]
print(f'La muestra representa el {round(mask_1_percent.sum(), 2)}% del total de los datos')
print(round(mask_1_percent, 2))

La muestra representa el 95.44% del total de los datos
Capital Federal                 26.66
Bs.As. G.B.A. Zona Norte        21.09
Bs.As. G.B.A. Zona Sur          11.51
Córdoba                          9.96
Santa Fe                         8.39
Buenos Aires Costa Atlántica     8.25
Bs.As. G.B.A. Zona Oeste         7.69
Buenos Aires Interior            1.89
Name: state_name, dtype: float64


In [388]:
# Comprobamos la cantidad de filas donde tengan valor nulo en todas las columnas y las eliminamos  
data_clean_empty_row = data.dropna(axis = 0, how= 'all')
print(data.shape)
print(data_clean_empty_row.shape)
#En este caso no hay ninguna fila con todos los valores nulos

(121220, 25)
(121220, 25)


In [389]:
# Con la mascara generada anteriormente para quedarnos con los state_names mayores a 1%, eliminamos las instancias que no corresponden a los state_names seleccionados
data = data[data.state_name.isin(mask_1_percent.index)]
data

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,-58.475596,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


## Analis preliminar del DataSet

In [390]:
# Usamos dtypes para saber que tipo de datos se almacenan en el dataset
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

In [391]:
#Vemos cuantos valores nulos tenemos por columna
is_null_result = data.isnull().sum()
print(is_null_result)

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18527
lat-lon                        48639
lat                            48639
lon                            48639
price                          17870
currency                       17871
price_aprox_local_currency     17870
price_aprox_usd                17870
surface_total_in_m2            37058
surface_covered_in_m2          17744
price_usd_per_m2               49058
price_per_m2                   30175
floor                         108073
rooms                          69989
expenses                      101758
properati_url                      0
description                        2
title                              0
image_thumbnail                 2848
dtype: int64


In [392]:
data_cero = (data==0).sum()
data_cero

operation                       0
property_type                   0
place_name                      0
place_with_parent_names         0
country_name                    0
state_name                      0
geonames_id                     0
lat-lon                         0
lat                             0
lon                             0
price                           1
currency                        0
price_aprox_local_currency      1
price_aprox_usd                 1
surface_total_in_m2           379
surface_covered_in_m2           2
price_usd_per_m2                0
price_per_m2                    0
floor                           0
rooms                           0
expenses                        0
properati_url                   0
description                     0
title                           0
image_thumbnail                 0
dtype: int64

In [393]:
#Vemos que porcentaje de nulos tiene cada columna
percentage_null = 100*is_null_result/data.shape[0]
percentage_null

operation                      0.000000
property_type                  0.000000
place_name                     0.019881
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                   16.014626
lat-lon                       42.043254
lat                           42.043254
lon                           42.043254
price                         15.446719
currency                      15.447583
price_aprox_local_currency    15.446719
price_aprox_usd               15.446719
surface_total_in_m2           32.032709
surface_covered_in_m2         15.337805
price_usd_per_m2              42.405435
price_per_m2                  26.083086
floor                         93.417641
rooms                         60.498064
expenses                      87.958993
properati_url                  0.000000
description                    0.001729
title                          0.000000
image_thumbnail                2.461794


## Tratamiento de nulos en la columna place_name

In [394]:
#Analizamos los nulos en la columna place_name con una mascara booleana
place_name = data.place_name
mask_place_name = place_name.isnull()
data_null_place_name = data[mask_place_name]
data_null_place_name

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,...,240.0,2708.333333,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,...,300.0,NaN,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...
11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261,...,270.0,253.456221,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...
14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174,...,200.0,2000.000000,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...
18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350,...,340.0,3814.705882,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...
21922,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.399704,-58.638405",-34.399704,-58.638405,...,220.0,NaN,2000.000000,NaN,5.0,NaN,http://www.properati.com.ar/17gy5_venta_casa_s...,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,https://thumbs4.properati.com/8/oO7kmFPYNm8DYr...
23664,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4005161998,-58.6417999185",-34.400516,-58.641800,...,294.0,1632.653061,1632.653061,NaN,NaN,NaN,http://www.properati.com.ar/17mb5_venta_casa_t...,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,https://thumbs4.properati.com/2/LQCwH6txpFqetn...
24722,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007292,-58.6377783",-34.400729,-58.637778,...,270.0,1677.215190,1962.962963,NaN,NaN,NaN,http://www.properati.com.ar/17oz1_venta_casa_t...,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/0/Nhx4NejZaQACap...
38856,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.425087,-58.5796585",-34.425087,-58.579659,...,550.0,NaN,2454.545455,NaN,NaN,NaN,http://www.properati.com.ar/18mcb_venta_casa_t...,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",https://thumbs4.properati.com/2/Izq8d5o8ti3kJy...
45970,sell,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.396074,-58.63738",-34.396074,-58.637380,...,27.0,1727.272727,3518.518519,NaN,1.0,NaN,http://www.properati.com.ar/1958f_venta_depart...,"EDIFICIO ZERENA –

In [395]:
# Como todos los nulos en la columna place_name corresponden al municipio de Tigre, los imputamos directamente con este municipio
data.loc[data_null_place_name.index, "place_name"] = 'Tigre'

## Analisis de columnas surface_total_in_m2 y surface_covered_in_m2

In [396]:
# Vemos en que casos el valor surface_total_in_m2 es menor que el valor surface_covered_in_m2
mask_surface = data[data.surface_total_in_m2 < data.surface_covered_in_m2]
mask_surface[["surface_total_in_m2" , "surface_covered_in_m2"]]

,surface_total_in_m2,surface_covered_in_m2
177,420.0,450.0
316,125.0,156.0
317,125.0,156.0
455,170.0,430.0
668,180.0,230.0
...,...,...
120168,120.0,140.0
120181,153.0,226.0
120202,70.0,200.0
120302,1200.0,12000.0


In [397]:
#En los casos en que surface_total_in_m2 sea menor que surface_covered_in_m2 imputamos los datos en el orden inverso

list_a = mask_surface.surface_total_in_m2
list_b = mask_surface.surface_covered_in_m2

data.loc[mask_surface.index, "surface_total_in_m2"] = list_b
data.loc[mask_surface.index, "surface_covered_in_m2"] = list_a

#### Datos de Superficie total o superficie cubierta que quedaran nulos o ceros por ahora

In [398]:
# Realizamos una mascara para ver las instancias de superficie tanto cubierta como total de los cuales no tenemos para imputar un valor con el otro

mask_surface_total_in_m2_zero = data[((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 == 0) | (data.surface_covered_in_m2.isnull()))]

print (mask_surface_total_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#En este DataFrame hay tanto superficies nulas como superficies iguales a cero

       property_type  surface_total_in_m2  surface_covered_in_m2
1          apartment                  NaN                    NaN
3                 PH                  NaN                    NaN
9              house                  NaN                    NaN
11             house                  NaN                    NaN
65             store                  NaN                    NaN
...              ...                  ...                    ...
121070     apartment                  NaN                    NaN
121079     apartment                  NaN                    NaN
121101         house                  NaN                    NaN
121134         house                  NaN                    NaN
121144         house                  NaN                    NaN

[11437 rows x 3 columns]


#### Datos de superficie total o superficie cubierta que puedo imputar

In [399]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie total pero que la cubierta no es nula ni cero, por lo tanto se la podemos imputar

mask_surface_total_in_m2_zero = data[((data.surface_total_in_m2 == 0) | (data.surface_total_in_m2.isnull())) & ((data.surface_covered_in_m2 != 0) & (data.surface_covered_in_m2.notnull()))]

print (mask_surface_total_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#Como el DataFrame no es nulo, vamos a poder imputar los valores nulos o ceros de superficie total con la superficie cubierta

       property_type  surface_total_in_m2  surface_covered_in_m2
68         apartment                  NaN                   72.0
77         apartment                  NaN                   85.0
79         apartment                  NaN                   42.0
87             house                  NaN                  170.0
89         apartment                  NaN                   53.0
...              ...                  ...                    ...
121155         house                  NaN                  130.0
121156         house                  NaN                  130.0
121207         house                  NaN                  178.0
121208         house                  NaN                  178.0
121210         house                  NaN                  120.0

[26000 rows x 3 columns]


In [400]:
# Realizamos una mascara para ver que valores tenemos nulos o ceros de superficie cubierta pero que la total no es nula ni cero, por lo tanto se la podemos imputar

mask_surface_covered_in_m2_zero = data[((data.surface_total_in_m2 != 0) | (data.surface_total_in_m2.notnull())) & ((data.surface_covered_in_m2 == 0) & (data.surface_covered_in_m2.isnull()))]

print (mask_surface_covered_in_m2_zero[["property_type" , "surface_total_in_m2" , "surface_covered_in_m2"]])

#Como el DataFrame es nulo, no vamos a poder imputar los valores nulos o ceros de superficie cubierta con la total

Empty DataFrame
Columns: [property_type, surface_total_in_m2, surface_covered_in_m2]
Index: []


In [401]:
# Aca imputamos los valores nulos o ceros de superficie total con los valores de superficie cubierta, utilizando la mascara mask_surface_total_in_m2_zero

data.loc[mask_surface_total_in_m2_zero.index, "surface_total_in_m2"] = data.loc[mask_surface_total_in_m2_zero.index, "surface_covered_in_m2"]

#Aca imputamos las instancias de superficie total iguales a cero o nulos con el valor correspondiente de esa instancia de superficie cubierta cuando este valor no es cero o nulo

## Analisis de columna rooms

### En las instancias que rooms sea igual a cero o nulo, imputamos el valor por medio de una expresion regular aplicada a la columna de descripcion. 

#### Funcion lamda a descripcion

In [402]:
# Aplicamos al DataSet una mascara booleana que contenga cero o nulo de rooms
boolean_mask_rooms = data[(data.rooms == 0) | (data.rooms.isnull())]
description_series = boolean_mask_rooms.description
description_series

0         2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...
1         Venta de departamento en décimo piso al frente...
2         2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3         PH 3 ambientes con patio. Hay 3 deptos en lote...
4         DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...
                                ...                        
121215    TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...
121216    Excelente e impecable casa en Venta en Las Lom...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121218    2 Amb al contrafrente,  luminoso. El departame...
121219    Departamento de 77 m2 apto profesional. Edific...
Name: description, Length: 69989, dtype: object

In [403]:
# Usamos expresiones regulares para sacar los rooms en descripcion
# Creamos un patron que encuentra numeros de un digito o mas, que pueden o no tener un espacio antes de la combinacion de letras amb indistindo de mayusculas o minusculas
#logica del patron: que tome un numero de un digito que solamente tenga detras uno o mas espacios y seguido de uno o mas espacios y las letras amb
patron = "(?P<rooms>\d+)\s+(?i)amb" 
pattern_nombre_regex = re.compile(patron)

In [404]:
# Mostramos donde hay matches
resultado_rooms = description_series.apply(lambda x: pattern_nombre_regex.search(str(x)))
print(resultado_rooms)

0         <re.Match object; span=(0, 5), match='2 AMB'>
1                                                  None
2         <re.Match object; span=(0, 5), match='2 AMB'>
3         <re.Match object; span=(3, 8), match='3 amb'>
4                                                  None
                              ...                      
121215                                             None
121216                                             None
121217                                             None
121218    <re.Match object; span=(0, 5), match='2 Amb'>
121219                                             None
Name: description, Length: 69989, dtype: object


In [405]:
# Extraemos los datos que usaremos para la imputación (pongo que x sea cero cuando es none, ya que si me trae el valor None, no lo puedo pasar a float)
rooms_match = resultado_rooms.apply(lambda x: 0 if x is None else x.group('rooms'))
print(rooms_match)

print("Rooms matcheados distintos de cero=",(rooms_match!=0).sum()) #aca veo que cantidad pudo matchear
print(rooms_match.unique()) #aca veo todos los rooms unicos que matcheo

0         2
1         0
2         2
3         3
4         0
         ..
121215    0
121216    0
121217    0
121218    2
121219    0
Name: description, Length: 69989, dtype: object
Rooms matcheados distintos de cero= 18608
['2' 0 '3' '1' '4' '7' '5' '6' '9' '2017' '103' '297' '072' '000' '11' '8'
 '085' '37' '800' '10' '12' '900' '0002' '400' '20174' '006' '17' '90'
 '0001' '772' '2004' '14' '24' '50' '20' '23' '65' '20173']


In [406]:
# Imputamos la serie rooms_match con su index en el DataSet 
data.loc[rooms_match.index, "rooms"] = rooms_match #((((Ver si esta imputacion esta bien asi))))
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment     0
2          apartment     2
3                 PH     3
4          apartment     0
...              ...   ...
121215     apartment     0
121216         house     0
121217     apartment     0
121218     apartment     2
121219     apartment     0

[115688 rows x 2 columns]


rooms    object
dtype: object

In [407]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

array([    2,     0,     3,     1,     4,     6,     5,    10,     7,
           9,     8,    17,    22,    15,    12,    11,    14,    16,
          20,    13,  2017,   103,   297,    72,    85,    25,    37,
         800,    19,    30,   900,    18,   400, 20174,    32,    90,
         772,  2004,    24,    50,    23,    31,    21,    29,    27,
          65,    28, 20173])

#### Funcion lamda a title

In [408]:
#Hacemos todo lo mismo que antes pero para obtener los rooms que se puedan de title
# Aplicamos al DataSet una mascara booleana que contenga cero rooms
boolean_mask_rooms_title = data[(data.rooms == 0)]
description_series_title = boolean_mask_rooms_title.title
description_series_title

1         VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37   ...
4         DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
6         MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...
7         JOSE HERNANDEZ 1400 MONOAMBIENTE  ESTRENAR CAT...
8         JOSE HERNANDEZ   1400 DOS AMBIENTES ESTRENAR ,...
                                ...                        
121214                                   Local - San Isidro
121215           Torre Forum Alcorta- Impecable 3 ambientes
121216    Ruca Inmuebles | Venta | Lomas de San Isidro |...
121217    VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...
121219           Departamento apto profesional en el Centro
Name: title, Length: 51394, dtype: object

In [409]:
resultado_rooms_title = description_series_title.apply(lambda x: pattern_nombre_regex.search(str(x)))
print(resultado_rooms_title)

1                                                    None
4          <re.Match object; span=(6, 11), match='2 AMB'>
6                                                    None
7                                                    None
8                                                    None
                               ...                       
121214                                               None
121215    <re.Match object; span=(31, 36), match='3 amb'>
121216                                               None
121217                                               None
121219                                               None
Name: title, Length: 51394, dtype: object


In [410]:
rooms_match_title = resultado_rooms_title.apply(lambda x: 0 if x is None else x.group('rooms'))
print(rooms_match_title)

print("Rooms matcheados distintos de cero=",(rooms_match_title != 0).sum()) #aca veo que cantidad pudo matchear
print(rooms_match_title.unique()) #aca veo todos los rooms unicos que matcheo

1         0
4         2
6         0
7         0
8         0
         ..
121214    0
121215    3
121216    0
121217    0
121219    0
Name: title, Length: 51394, dtype: object
Rooms matcheados distintos de cero= 2518
[0 '2' '6' '3' '4' '5' '9' '1' '7' '8' '15' '4000' '360' '12']


In [411]:
# Imputamos la serie rooms_match_title con su index en el DataSet 
data.loc[rooms_match_title.index, "rooms"] = rooms_match_title #((((Ver si esta imputacion esta bien asi))))
print(data[["property_type" , "rooms"]])
data[["rooms"]].dtypes

       property_type rooms
0                 PH     2
1          apartment     0
2          apartment     2
3                 PH     3
4          apartment     2
...              ...   ...
121215     apartment     3
121216         house     0
121217     apartment     0
121218     apartment     2
121219     apartment     0

[115688 rows x 2 columns]


rooms    object
dtype: object

In [412]:
 #Cambio el tipo de dato de object a float para homogeneizarlo con los datos numericos que tenemos
data["rooms"] = data["rooms"].astype("int")
data[["rooms"]].dtypes
data.rooms.unique() #Aca quiero ver que valores unicos hay de rooms(acompanarlo con un grafico seria lo adecuado)

array([    2,     0,     3,     1,     4,     6,     5,    10,     7,
           9,     8,    17,    22,    15,    12,    11,    14,    16,
          20,    13,  2017,   103,   297,    72,    85,    25,    37,
         800,    19,    30,   900,    18,   400, 20174,  4000,    32,
          90,   772,  2004,    24,    50,   360,    23,    31,    21,
          29,    27,    65,    28, 20173])

In [413]:
#Aca veo, de los rooms que son iguales a cero, cuales tienen la superficie cubierta distinta de cero o nulo para imputarla despues
data_rooms_surface = data[(data.rooms == 0) & ((data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0) | (data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0))]
data_rooms_surface[["rooms" , "surface_covered_in_m2"]]

,rooms,surface_covered_in_m2
6,0,78.0
7,0,40.0
8,0,60.0
18,0,270.0
20,0,148.0
...,...,...
121213,0,600.0
121214,0,123.0
121216,0,360.0
121217,0,39.0


#### Vemos instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

In [424]:
#Aca veolas instancias que tengan el precio y el precio por metro cuadrado pero no la superficie

data_price_price_m2 = data[((data.price != 0) & (data.price.notnull())) & ((data.price_usd_per_m2.notnull() & data.price_usd_per_m2!=0) | (data.price_per_m2.notnull() & data.price_per_m2!=0))]
data_rooms_surface[["price" , "price_usd_per_m2" , "price_per_m2"]]

,price,price_usd_per_m2,price_per_m2
6,130000.0,1226.415094,1666.666667
7,138000.0,3066.666667,3450.000000
8,195000.0,3000.000000,3250.000000
18,570000.0,2111.111111,2111.111111
20,NaN,NaN,NaN
...,...,...,...
121213,NaN,NaN,NaN
121214,NaN,NaN,NaN
121216,498000.0,1383.333333,1383.333333
121217,131500.0,2858.695652,3371.794872


# **VENIMOS CODEANDO HASTA ACA, NO DAR BOLA A LO QUE SIGUE**

In [414]:

# Queremos, para los que tengan ambas superficies nulas o cero se las imputamos por una media condicionada por el valor de property_type, place_name, state_name y rooms

In [415]:
# Vemos cuantos unique hay en place_name para ver que no exitan dos place_names iguales para dos state_names diferentes (ej. centro)
data.place_name.unique()

array(['Mataderos', 'La Plata', 'Liniers', 'Centro', 'Munro', 'Belgrano',
       'Rosario', 'Córdoba', 'San Miguel', 'Martínez', 'Palermo Soho',
       'Bella Vista', 'Barrio El Golf', 'Tigre', 'Palermo',
       'Villa Giardino', 'General Rodríguez', 'Pilar', 'Castelar',
       'Barrio San Gabriel', 'Estancias del Pilar', 'Flores', 'Boedo',
       'Malagueño', 'Las Cañitas', 'Del Viso', 'Villa Carlos Paz',
       'Puerto Madero', 'Balvanera', 'Escobar', 'San Isidro', 'Caballito',
       'Acassuso', 'Nuñez', 'Fincas de Iraola', 'Berazategui',
       'San Martín', 'Ituzaingó', 'Mar del Plata', 'Caseros',
       'San Bernardo', 'San Telmo', 'Almagro', 'Nordelta',
       'Capital Federal', 'Boulogne Sur Mer', 'Victoria', 'Pinamar',
       'Benavidez', 'Beccar', 'Luján', 'San Lorenzo',
       'Bs.As. G.B.A. Zona Sur', 'Lomas del Mirador', 'Lanús Este',
       'Punilla', 'Villa Gesell', 'Banfield', 'Jose Leon Suarez',
       'Olivos', 'Santa Fe', 'Colegiales', 'Exaltación de la Cruz',
      

In [416]:
# Vemos los que tienen rooms nulos y superficies nulas
mas_rooms = (data.rooms.isnull() & data.surface_covered_in_m2.isnull() & data.surface_total_in_m2.isnull()) or (data.rooms == 0 & data.surface_covered_in_m2 == 0 & data.surface_total_in_m2.isnull == 0 & data.surface_total_in_m2 == 0)
data_rooms = data[mask_rooms]
data_rooms[["property_type", "place_name" , "state_name" ,"rooms", "surface_covered_in_m2" , "surface_total_in_m2" ]]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [374]:
#Aca hago una maquinita manual para obtener el numero de instancia de los valores raros de rooms
marcaradedatos =  data[data.rooms == "0002"]
marcaradedatos.description[43292]

'DEPARTAMENTOS A ESTRENAR FINANCIADOSMonoambiente: dormitorio-cocina separada-balcón dde $750.0002 Ambientes: living comedor-cocina-dormitorio-baño completo-balcón dde $ 1.230.0003 Ambientes: living comedor-cocina separada-2 dormotorios-baño -toilette-terraza dde $ 2.000.000. EasyBroker ID: EB-AU9109'

In [ ]:
#Aca empiezo a tomar una muestra para ver que pasa

In [319]:
data.description[117551       ]

'                        Edificio de categoría,a estrenar con amenities, piscina ,solarium y gim equipado, en inmejorable zona de Cap. Fed. Entrega junio de 20173 Ambientes al frente todo luz y sol con cochera ( numero 22 ) . Living comedor , dos dormitorios en suite el principal con vestidor, Cocina integrada, con espacio para lavarropas. . Excelente ubicación Seguí entre Ugarteche y Ortiz, a una cuadra de Av. Libertador y 2 de Av. Las Heras.'

In [248]:
data.description[1601]

'Corredor Responsable: Matiaz Ruiz Moreno - CPCPI 4740Contacto: Aldo González - MLS ID # 420581059-22Vendo Casa en La Granja, Departamento Colon en la Provincia de Córdoba.  Ubicada en excelente barrio y entorno. Con muy buena vista a las sierras y cercana al río,  lo que da un alto rendimiento para la inversión,  con fines de alquiler por temporada o residencia permanente, rodeadas de casas de categoría y complejos turísticos de nivel. A solo 10 minutos de Estancia La Paz y Ascochinga Golf Club.  La casa esta compuesta por dos plantas.  La planta alta y principal tiene un hall de ingreso, amplio living comedor, cocina con alacenas y muebles de bajo mesada, separada por un desayunador. Cuatro dormitorios con amplios placares, dos de ellos con baño en suite, mas dos baños de uso común. Gran galería con asador y una imponente vista.  La planta baja tiene ingreso independiente y esta compuesta por 4 dormitorios, un baño, lavadero, living comedor de importante tamaño y galería con acceso a

In [249]:
data.description[116017]

'EXCELENTE CASA EN VILLA GENERAL BELGRANO APTA CREDITOLIVING COMEDOR AMPLIO CON ESTUFA HOGAR\xa0COCINA AMOBLADALAVADERODESAYUNADOR\xa03 DORMITORIOS CON PLACARES UNO DE ELLOS EN SUITE CON ANTEBAÑOBAÑO CON ANTEBAÑOPLAY ROOM AMPLISIMO CON BALCON\xa0CUARTO DE HERRAMIENTAS\xa0COCHERA DOBLE NO PASANTE\xa0QUINCHO GALERIA CON ASADOR BAJOASADOR\xa0PISICNA\xa0HERMOSA VISTA A LAS SIERRAS\xa0ZONA DE GRAN TRANQUILIDADCONSULTAS AL 3512364585\xa0'

In [251]:
data.description[76675	]

'SE VENDE URGENTE -Hermoso monoambiente en exclusiva zona de Las Cañitas - Palermo .Apto CreditoA mts. de Av.Libertador, cantidad de lineas de colectivos , del Hipódromo Argentino de  Palermo, del polo gastronomico sobre Baez y Av.Gral. Cheanult ,a 700m2  de Av. Dorrego y del Shopping.Ideal Inversor - Zona turística por excelencia -Alquiler apoximadamente en pesos por es $12.0001 ambiente contrafrente ,con cocina electrica , muebles bajo y sobre mesada, baño completo ,placard separado bastante amplio.ROILANDS REAL ESTATE NETWORKAv. Libertador 6810 4to. A, CABA, Tel.  Corredor responsable: Mariano Mussi C.U.C.I.C.B.A Nro. 4065Av. Libertador San Martín 23, Los Cardales, Tel. Corredor responsable: Fabian Jose Mussi C.M.Z.C Nro. 217 T.1 F.73'

In [245]:
data.description[7689]

'EXCELENTE DEPARTAMENTO MONOAMBIENTE A ESTRENAR ENTREGA SEPTIEMBRE DE 2017 AMBIENTE UNICO CON COCINA INCORPORADA, BAÑO COMPLETO, BALCON AL CONTRAFRENTE MUY LUMINOSO, EDIFICIO CON SUM, LAUNDRY, PISCINA Y PARRILLA. -  - BEZ PROPIEDADES'

## Analisis de columna price

### Intentaremos imputar las instancias de precios nulas o iguales a cero con expresiones regulares en la columna descripcion